In [53]:
import os
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import numpy as np
import glob
from datetime import datetime
import xlrd

In [54]:
# TODO: Set countryXpath and dataTypeXpath and set the beginning of the file name

# Ecuador
country_xpath = '//*[@id="otherCountries"]/div[1]/div[1]/div/div/ul/li[13]/a'

# Export Declarations
data_xpath = '//*[@id="otherCountries"]/div[1]/div[2]/div/div/ul/li[3]/a'

# Beginning of File Name
file_beg = "EcuadorExports_"

In [55]:
def create_chrome_driver(download_path):
    # set executable path
    chrome_driver = "/usr/local/bin/chromedriver"
    os.environ["webdriver.chrome.driver"] = chrome_driver
    
    # set default download directory
    options = webdriver.ChromeOptions()
    prefs = {"download.default_directory" : download_path,
             "profile.default_content_setting_values.automatic_downloads": 1
            }
    options.add_experimental_option("prefs",prefs)

    return webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)

In [56]:
def datamyne_login(driver):
    driver.get("https://threezero.datamyne.com/frontend/loadSearchForm.thyme?appId=1731&ticket=dcc1e1f:1743f37103e:-63e3")
    input_user = driver.find_element_by_name("user")
    input_user.send_keys("Michigan")
    input_pw = driver.find_element_by_name("password")
    input_pw.send_keys("Primavera1")
    input_pw.send_keys(Keys.RETURN)

In [57]:
def query(driver):
    country_dropdown = driver.find_element(By.CSS_SELECTOR, '#otherCountries > div:nth-child(1) > div:nth-child(1) > div > button')
    country_dropdown.click()
    
    # Select Country
    country = driver.find_element(By.XPATH, country_xpath)
    country.click()
    
    data_dropdown = driver.find_element(By.CSS_SELECTOR, 'button[data-id="select-data"]')
    data_dropdown.click()
    
    # Select Data Type
    data = driver.find_element(By.XPATH, data_xpath)
    data.click()
    
    query_button = driver.find_element(By.ID, 'queries_button')
    query_button.click()
    
    driver.implicitly_wait(30)

In [58]:
def select_date(driver, start, end):
    wait = WebDriverWait(driver, 10)
    input_date = wait.until(EC.element_to_be_clickable((By.ID, 'searchRanges')))
    input_date.clear()
    date_range = start + " - " + end
    input_date.send_keys(date_range)

    apply = driver.find_element(By.CLASS_NAME, 'applyBtn')
    apply.click()

In [59]:
def download_data(driver, download_path, start, start_date, end_date):
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, 'dm-modal-backdrop')))
    
    # Click Random Button
    rand = driver.find_element(By.CLASS_NAME, 'rowCheckbox')
    rand.click()        
    rand.click()
        
    # Click Download button
    wait2 = WebDriverWait(driver, 10)
    excel = wait2.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#btnExcel > button:nth-child(1)')))
    excel.click()
        
    # Set File Name
    file_name = driver.find_element(By.ID, 'excelNameSave')
    file_name.clear()
    
    name = file_beg + start_date + "_" + end_date + "_" + str(start) + "_" + str(start + 4999)
    file_name.send_keys(name)
        
    # Specify Range
    export_range = driver.find_element(By.ID, 'radioToExportRange')
    export_range.click()
    start_range = driver.find_element(By.ID, 'dd_startRangeE')
    start_range.send_keys(str(start))
    end_range = driver.find_element(By.ID, 'dd_endRangeE')
    end_range.send_keys(str(start + 4999))
    export = driver.find_element(By.CLASS_NAME, 'excelLoad')
    export.click()
        
    return start + 5000

In [60]:
def get_last_date(download_path):
    # get most recently downloaded file
    file_pattern = "*.xlsx"
    list_of_files = glob.iglob(download_path + file_pattern)
    latest_file = max(list_of_files, key=os.path.getctime)
    
    # get last date
    wb = xlrd.open_workbook(latest_file) 
    sheet = wb.sheet_by_index(0) 
    
    # format date
    excel_date = int(sheet.cell_value(5000, 0))
    python_date = datetime(*xlrd.xldate_as_tuple(excel_date, 0))
    return python_date

In [61]:
def download_hundred_thousand(driver, download_path, start_date):
    end_date = get_last_date(download_path).strftime('%m/%d/%Y')
    start_name = start_date.replace("/", "_")
    end_name = end_date.replace("/", "_")
    select_date(driver, start_date, end_date)
    
    start = 1
    while (start < 100000):
        try:
            start = download_data(driver, download_path, start, start_name, end_name)
            time.sleep(5)
        except:
            time.sleep(10)
            pass
    return end_date

In [65]:
def first_time(download_path, start_date, end_date):
    driver = create_chrome_driver(download_path)
    datamyne_login(driver)
    query(driver)
    
    time.sleep(7)
    
    start_name = start_date.replace("/", "_")
    end_name = end_date.replace("/", "_")
    select_date(driver, start_date, end_date)
    
    start = 1
    while (start < 100000):
        try:
            start = download_data(driver, download_path, start, start_name, end_name)
            time.sleep(5)
        except:
            time.sleep(10)
            pass
        
    time.sleep(60)
    driver.close()

In [67]:
# TODO: Before downloading data for each year, 
# 1. Set download path
# 2. Set starting and ending dates (should just be 01/01/xx and 12/31/xx)

download_path = "/Users/rachelye/Desktop/Ecuador Exports 2018/"
start_date = "01/01/2018"
end_date = "12/31/2018"
first_time(download_path, start_date, end_date)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/rachelye/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [74]:
def run(download_path, n, start_date):
    driver = create_chrome_driver(download_path)
    datamyne_login(driver)
    query(driver)
    
    time.sleep(15)
    
    for i in range(n):
        download_hundred_thousand(driver, download_path, start_date)
        time.sleep(30)
    
    time.sleep(60)
    driver.close()

In [75]:
# TODO: Run all the cells above this one, and then 
# run only this cell until all files have been downloaded

run(download_path, 10, start_date)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/rachelye/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


KeyboardInterrupt: 

In [42]:
def find_missing(download_path):
    file_pattern = "*.xlsx"
    files = glob.iglob(download_path + file_pattern)

    dates = {}
    for f in files:
        file_name = f.split("_")[1:]
        start_date = file_name[0] + "/" + file_name[1] + "/" + file_name[2]
        end_date = file_name[3] + "/" + file_name[4] + "/" + file_name[5]
        start_row = int(file_name[6])

        if end_date in dates:
            dates[end_date].add(start_row)
        else:
            dates[end_date] = set()
            dates[end_date].add(start_row)
    
    # values that should be present
    s = set()
    for i in range(20):
        s.add(i * 5000 + 1)
    
    missing = {}
    for d in dates:
        last = max(dates[d])
        diff = set()
        for d1 in s.difference(dates[d]):
            if d1 < last:
                diff.add(d1)
        if (diff != set()):
            missing[d] = diff
           
    return missing

In [43]:
def replace_missing(download_path, start_date):
    missing = find_missing(download_path)
    print(missing)
    if (missing == {}):
        return
    
    driver = create_chrome_driver(download_path)
    datamyne_login(driver)
    query(driver)
    
    time.sleep(7)
    
    for m in missing:
        start_name = start_date.replace("/", "_")
        end_name = m.replace("/", "_")
        select_date(driver, start_date, m)
        for start in missing[m]:
            s = start
            while (s == start):
                try:
                    s = download_data(driver, download_path, start, start_name, end_name)
                    time.sleep(5)
                except:
                    time.sleep(10)
                    pass
    time.sleep(60)
    driver.close()

In [44]:
# Run this cell to make sure you have correctly downloaded all the files
replace_missing(download_path, start_date)

[WDM] - Current google-chrome version is 86.0.4240


{'12/31/2019': {75001}}
 


[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/rachelye/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [45]:
# Run this cell just to make sure there is nothing missing
replace_missing(download_path, start_date)

{}
